In [1]:
import datetime
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings("ignore")

In [2]:
users = pd.read_csv('takehome_users.csv', encoding='latin_1')
engagement = pd.read_csv('takehome_user_engagement.csv')

In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [4]:
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [5]:
#renaming object_id column
users = users.rename({"object_id":"user_id"}, axis=1)

In [6]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [7]:
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [8]:
print(users.isna().sum())
print(engagement.isna().sum())

user_id                          0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
dtype: int64
time_stamp    0
user_id       0
visited       0
dtype: int64


#### Problem:
Defining an "adopted user" as a user who has logged into the product on three separate
days in at least one seven day period, identify which factors predict future user adoption.

We know that an "adopted user" is one who has logged in 3 times, so if a user doesn't have a last session, they certainly don't have 2 prior to that. We can remove these users. 

In [9]:
users = users.dropna(subset=['last_session_creation_time'])
users.isna().sum()

user_id                          0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time       0
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            4047
dtype: int64

In [10]:
#converting columns to time stamps
engagement['time_stamp'] = pd.to_datetime(engagement['time_stamp'], format= '%Y-%m-%d %H:%M:%S')
users['creation_time'] = pd.to_datetime(users['creation_time'], format='%Y-%m-%d %H:%M:%S')
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'], unit='s', origin='unix')

In [11]:
# defining an adopted user
by_times = engagement.set_index("time_stamp")

clients = by_times["user_id"].unique()
adoption = []

for i in clients:
    by_id = by_times["user_id"] == i
    df_filter = by_times[by_id].resample("1D").count()
    df_filter = df_filter.rolling(window=7).sum()
    df_filter = df_filter.dropna()
    adoption.append(any(df_filter["visited"].values >= 3))

In [12]:
#making booleans for adopted users into a dataframe
user_adoption = list(zip(clients, adoption))

df_adopt = pd.DataFrame(user_adoption)
df_adopt.head()

,0,1
0,1,False
1,2,True
2,3,False
3,4,False
4,5,False


In [13]:
#re-name columns
df_adopt.columns = ["user_id", "adopted_user"]

#merge two dataframes together so all info is in one place
df = users.merge(df_adopt, on="user_id", how="outer")
df.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,False
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,True
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,False
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,False
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,False


In [14]:
#changing booleans to 0/1s
df.loc[:, "adopted_user"] = df["adopted_user"].map({True:1, False:0, np.nan:0})

In [15]:
df['adopted_user'].value_counts()

0    7226
1    1597
Name: adopted_user, dtype: int64

In [16]:
#how many unique users are there and making sure original number matches merged dataframe
print('number of unique users:', len(users['user_id'].unique()))
len(users['user_id'].unique()) == len(df['user_id'].unique())

number of unique users: 8823


True

In [17]:
print('sources:', df.creation_source.unique())

sources: ['GUEST_INVITE' 'ORG_INVITE' 'SIGNUP' 'PERSONAL_PROJECTS'
 'SIGNUP_GOOGLE_AUTH']


In [18]:
#assigning dummy variables to creation_source
sources = ['GUEST_INVITE', 'ORG_INVITE', 'SIGNUP', 'PERSONAL_PROJECTS', 'SIGNUP_GOOGLE_AUTH']
dummies = pd.get_dummies(df['creation_source'])

#adding dummy columns to df
df = pd.concat([df, dummies], axis=1)
df.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,GUEST_INVITE,ORG_INVITE,PERSONAL_PROJECTS,SIGNUP,SIGNUP_GOOGLE_AUTH
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,0,1,0,0,0,0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,1,0,1,0,0,0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,0,0,1,0,0,0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,0,1,0,0,0,0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,0,1,0,0,0,0


In [19]:
#looking for na values
df.isna().sum()

user_id                          0
creation_time                    0
name                             0
email                            0
creation_source                  0
last_session_creation_time       0
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            4047
adopted_user                     0
GUEST_INVITE                     0
ORG_INVITE                       0
PERSONAL_PROJECTS                0
SIGNUP                           0
SIGNUP_GOOGLE_AUTH               0
dtype: int64

In [20]:
#changing na values for invited_by to 0 to keep these rows 
#if any other features are more predictive and should be counted
df['invited_by_user_id'].fillna(value=0, inplace=True)

In [21]:
#scaling the data to standardize it
features = ['opted_in_to_mailing_list', 'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id',
           'GUEST_INVITE', 'ORG_INVITE', 'SIGNUP', 'PERSONAL_PROJECTS', 'SIGNUP_GOOGLE_AUTH']
X = df.loc[:, features].values
y = df.loc[:, ['adopted_user']].values

X = StandardScaler().fit_transform(X)

In [22]:
pca = PCA()
principal_components = pca.fit_transform(X)

principal_df = pd.DataFrame(data= principal_components)
principal_df.head()

,0,1,2,3,4,5,6,7,8
0,1.695140,0.676110,2.537491,-0.063435,-0.136577,-1.049968,-1.511408,-0.741872,-5.507896e-15
1,0.570848,-0.623279,-1.196504,0.062214,-0.073557,-1.062642,0.137269,1.272809,1.044190e-14
2,0.736320,-0.630359,-1.149196,0.016428,-0.026137,-0.316718,0.130524,1.013569,7.099812e-16
3,0.741133,-0.911312,2.215895,-0.094918,-0.176987,-1.126456,0.130580,0.345721,2.474278e-14
4,0.693231,-0.913625,2.239044,-0.163572,-0.055680,0.411533,0.122843,0.284436,1.813230e-14


In [23]:
#explained variance
sum(pca.explained_variance_ratio_[0:6])

0.9139009525212676

Looks like the first 6 components make up about 91% of the variation in the data. 

In [24]:
columns = features
feature_df = pd.DataFrame(pca.components_,columns=columns,
                          index = ['PC-1','PC-2', 'PC-3', 'PC-4', 'PC-5', 'PC-6', 'PC-7', 'PC-8', 'PC-9'])
feature_df.head()

,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,GUEST_INVITE,ORG_INVITE,SIGNUP,PERSONAL_PROJECTS,SIGNUP_GOOGLE_AUTH
PC-1,0.018502,0.011525,-0.040293,0.629166,0.188490,0.529764,-0.400418,-0.193013,-0.297126
PC-2,0.702055,0.703333,-0.001202,-0.019873,-0.072416,0.035944,0.063353,-0.035655,-0.014985
PC-3,0.063218,0.059290,0.013189,0.120364,0.823308,-0.537865,-0.094294,-0.019206,-0.037835
PC-4,0.041591,0.036678,-0.043781,-0.041789,-0.043866,-0.004334,-0.681397,0.142008,0.711934
PC-5,0.038660,0.014711,0.079011,-0.037809,-0.027738,-0.007587,-0.223743,0.886145,-0.393035


In [25]:
#adding up correlations 
abs(feature_df.head(6).sum(axis=0)).sort_values(ascending=False)

SIGNUP                        1.372139
org_id                        1.001518
GUEST_INVITE                  0.872967
opted_in_to_mailing_list      0.860436
enabled_for_marketing_drip    0.829641
PERSONAL_PROJECTS             0.702529
invited_by_user_id            0.653262
ORG_INVITE                    0.054981
SIGNUP_GOOGLE_AUTH            0.012259
dtype: float64

If we add up all the signup types, that is clearly the biggest feature, since the individual creation_source values come in at numbers 1, 3, 6, 7, and 8. The next most predictive feature would seem to be which organization the user belongs to (org_id). It would be interesting to run this through a training and testing scenario to see if these hold true for predicting adopted users. 